<a href="https://colab.research.google.com/github/CristianRitcher/Stoneheart/blob/main/image_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
print(tf.__version__)

2.17.0


In [ ]:
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_dir = 'data_0'
img_ext = ['jpeg', 'jpg', 'png', 'bmp']

In [ ]:
for img_class in os.listdir(data_dir):
  print(img_class)

sad_face
happy_face
.ipynb_checkpoints


In [ ]:
for img_class in os.path.join(data_dir):
  for image in os.listdir(os.path.join(data_dir, img_class)):
    img_path = os.path.join(data_dir, img_class, image)
    try:
      img = cv2.imread(img_path)
      tip = imghdr.what(img_path)
      if tip not in img_ext:
        print('Imgage not in ext list {}'.format(img_path))
        os.remove(img_path)
    except Exception as e:
      print( 'Issue with image {}'. format(img_path))

### 2. Descargar datos

In [ ]:
#tf.data.Dataset.list_files
#tf.keras.utils.image_dataset_from_directory()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data_0')
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
# permite ver el index asociado a la imagen
# fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# for idx, img in enumerate(batch[0][:4]):
  # ax[idx]. imshow(img. astype(int))
  # ax[idx].title.set_text(batch[1][idx])

## 3. Preprocesamiento

Reescalado


In [ ]:
data = data.map(lambda x, y: (x/255, y))
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

Split data

In [ ]:
# tamaños de muestra, checar longitud de batches so len(data) = train_size + val_size + test_size
train_size = int(len(data) * .7)
val_size = int(len(data) * .2)
test_size = int(len(data) * .1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

### 4. Modelo

Construcción del modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
#model.summary


Entrenamiento

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epchs=20, validation_data=val, callbacks=[tensorboard_callback])

Plot performance


In [ ]:
#checamos pérdida, forma validada:  logarítmica negativa
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', lebel='loss')
plt.plot(hist.history['val_loss'], color='orange', lebel='val_loss')
fig.subtitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
#checamos accuracy forma validada:  radical
plt.plot(hist.history['accuracy'], color='teal', lebel='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', lebel='val_accuracy')
fig.subtitle('accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.show()

### 5. Evaluación del performance

evaluación

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy

In [ ]:
for batch in test.as_numpy_iterator():
  X, y = batch
  yhat = model.predict(X)
  pre.update_state(y, yhat)
  re.update_state(y, yhat)
  acc.update_state(y, yhat)

In [ ]:
print(f'presición: {pre.result().numpy()}, Recall {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

test


In [ ]:
#Carga y muestra la imagen
img = cv2.imread('nombre_imagen.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show

In [ ]:
# la pone en el tamaño adecuado
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))
#predice el valor de 0 a 1 (siguiendo la sigmoidea)

### 6. Guardar el modelo

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#Guarda un modelo
model.save(os.path.join('models', 'nombremodelo.h5'))

In [ ]:
# carga un modelo
new_model = load_model(os.path.join('models', 'nombremodel.h5'))

In [ ]:
# usa el modelo
new_model.predict(np.expand_dims(resize/255, 0))